In [25]:
import sagemaker
import boto3
import pandas
import numpy
import io
import matplotlib.pyplot as plt

In [17]:
# If you are running the script locally uncomment this line
# role = "arn-sagemaker-role-for-the-user"

# If you are running this notebook in Sagemaker Notebook, uncomment this line
# role = sagemaker.get_execution_role()

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

endpoint_name_1 = "loci-v8-1h-endpoint"
# Get the model_package_arn
modelpackage_arn = ""

# create a deployable model for damage inspection model package.
model = sagemaker.ModelPackage(
    role=role,
    model_package_arn=modelpackage_arn,
    sagemaker_session=sagemaker_session,
)

In [ ]:
# Deploy the model
predictor = model.deploy(1, "ml.g5.xlarge", endpoint_name=endpoint_name_1)

In [ ]:
data = pandas.read_csv('../data/arm_dataset.csv')
print(data.head())
data_bytes = data.to_csv(None, index=False)

In [ ]:
def plot_distribution(weights):
    x = numpy.linspace(0, 10, 128)
    plt.bar(x, weights)
    plt.title('Cycle Distribution')
    plt.xlabel('Log Cycle time')
    plt.ylabel('Probability Density')
    plt.grid(True)
    plt.show()

In [ ]:
# Perform a prediction
response = runtime.invoke_endpoint(
    EndpointName=endpoint_name_1,
    ContentType="text/csv",
    Body=data_bytes
)
array_dist = numpy.load(io.BytesIO(response['Body'].read()))
# View the predictiona
plot_distribution(array_dist[0])

sample_log_cycle_time = numpy.random.choice(
    a=numpy.linspace(0, 10, 128),
    size=10000,
    p=array_dist[0]
)
avg_cycle_time = int(numpy.exp(numpy.mean(sample_log_cycle_time)))
stddev_cycle_time = int(numpy.exp(numpy.std(sample_log_cycle_time)))
median_cycle_time = int(numpy.exp(numpy.median(sample_log_cycle_time)))
min_cycle_time = int(numpy.exp(numpy.min(sample_log_cycle_time)))
max_cycle_time = int(numpy.exp(numpy.max(sample_log_cycle_time)))

report = (
    f"Average cycle time: {avg_cycle_time}\n" \
    f"Standard deviation: {stddev_cycle_time}\n" \
    f"Median cycle time: {median_cycle_time}\n" \
    f"Minimum cycle time: {min_cycle_time}\n" \
    f"Maximum cycle time: {max_cycle_time}\n" \
)
print(report)

In [ ]:
# Delete the endpoint and the model
model.sagemaker_session.delete_endpoint(endpoint_name_1)
model.sagemaker_session.delete_endpoint_config(endpoint_name_1)
model.delete_model()